In [4]:
import os
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime
from pandas.api.types import CategoricalDtype
import sqlite3
pd.set_option('display.max_columns', 500)


## Model #2

Now let's predict the WM 2014

In [10]:
prepared_matches = pd.read_csv("prepared_matches.csv", low_memory = False)
prepared_matches = prepared_matches.drop(["Unnamed: 0"],  axis=1)

In [5]:
prepared_matches_train = pd.read_csv("prepared_matches_train.csv", low_memory = False)
prepared_matches_train = prepared_matches_train.drop(["Unnamed: 0"],  axis=1)
prepared_matches_train.tail(10)

,id_int,id_int_home,id_int_away,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,class_AFC Asian Cup 2007,class_AFC Asian Cup 2011,class_AFC Asian Cup 2015,class_AFC Challenge Cup 2014,class_AFC-OFC Challenge Cup,class_Afrika-Cup 1998,class_Afrika-Cup 2000,class_Afrika-Cup 2002,class_Afrika-Cup 2004,class_Afrika-Cup 2006,class_Afrika-Cup 2008,class_Afrika-Cup 2010,class_Afrika-Cup 2012,class_Afrika-Cup 2013,class_Afrika-Cup 2015,class_Afrika-Cup 2017,class_Afrika-Cup-Qualifikation,class_Artemio Franchi Trophy,class_Asian Cup Qualifikation,class_CONCACAF Cup,class_Caribbean Cup 2005,class_Caribbean Cup 2007,class_Caribbean Cup 2008,class_Caribbean Cup 2010,class_Caribbean Cup 2012,class_Caribbean Cup 2014,class_Caribbean Cup 2017,class_Caribbean Cup Qualifikation,class_Confederations Cup 1997,class_Confederations Cup 1999,class_Confederations Cup 2001,class_Confederations Cup 2003,class_Confederations Cup 2005,class_Confederations Cup 2009,class_Confederations Cup 2013,class_Confederations Cup 2017,class_Copa América 1983,class_Copa América 1987,class_Copa América 1989,class_Copa América 1991,class_Copa América 1993,class_Copa América 1995,class_Copa América 1997,class_Copa América 1999,class_Copa América 2001,class_Copa América 2004,class_Copa América 2007,class_Copa América 2011,class_Copa América 2015,class_Copa América Centenario 2016,class_Copa América Centenario Play-In,class_Copa Centroamericana 2011,class_Copa Centroamericana 2013,class_Copa Centroamericana 2014,class_Copa Centroamericana 2017,class_EM-Qualifikation,class_EM-Qualifikation Playoffs,class_Europameisterschaft 1980,class_Europameisterschaft 1984,class_Europameisterschaft 1988,class_Europameisterschaft 1992,class_Europameisterschaft 1996,class_Europameisterschaft 2000,class_Europameisterschaft 2004,class_Europameisterschaft 2008,class_Europameisterschaft 2012,class_Europameisterschaft 2016,class_Freundschaftsspiele,class_Gold Cup 1991,class_Gold Cup 1993,class_Gold Cup 1996,class_Gold Cup 1998,class_Gold Cup 2000,class_Gold Cup 2002,class_Gold Cup 2003,class_Gold Cup 2005,class_Gold Cup 2007,class_Gold Cup 2009,class_Gold Cup 2011,class_Gold Cup 2013,class_Gold Cup 2015,class_Gold Cup 2017,class_Gold Cup Playoff,class_Gold Cup Qualifikation,class_König-Fahd-Pokal 1992,class_König-Fahd-Pokal 1995,class_OFC Nations Cup 1996,class_OFC Nat

In [7]:
#get all games during WM 2014, print the last two games (Finale, Spiel um Platz 3).
matches_wm2014 = prepared_matches_train[(1432 < prepared_matches_train.game_days_before_wm) & (prepared_matches_train.game_days_before_wm < 1465)]

prepared_matches_train_without_after_wm14 = prepared_matches_train[prepared_matches_train.game_days_before_wm < 1465]

prepared_matches_train_without_after_wm14.shape

(2980, 289)

In [8]:
matches_wm2014.tail(2)

,id_int,id_int_home,id_int_away,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,class_AFC Asian Cup 2007,class_AFC Asian Cup 2011,class_AFC Asian Cup 2015,class_AFC Challenge Cup 2014,class_AFC-OFC Challenge Cup,class_Afrika-Cup 1998,class_Afrika-Cup 2000,class_Afrika-Cup 2002,class_Afrika-Cup 2004,class_Afrika-Cup 2006,class_Afrika-Cup 2008,class_Afrika-Cup 2010,class_Afrika-Cup 2012,class_Afrika-Cup 2013,class_Afrika-Cup 2015,class_Afrika-Cup 2017,class_Afrika-Cup-Qualifikation,class_Artemio Franchi Trophy,class_Asian Cup Qualifikation,class_CONCACAF Cup,class_Caribbean Cup 2005,class_Caribbean Cup 2007,class_Caribbean Cup 2008,class_Caribbean Cup 2010,class_Caribbean Cup 2012,class_Caribbean Cup 2014,class_Caribbean Cup 2017,class_Caribbean Cup Qualifikation,class_Confederations Cup 1997,class_Confederations Cup 1999,class_Confederations Cup 2001,class_Confederations Cup 2003,class_Confederations Cup 2005,class_Confederations Cup 2009,class_Confederations Cup 2013,class_Confederations Cup 2017,class_Copa América 1983,class_Copa América 1987,class_Copa América 1989,class_Copa América 1991,class_Copa América 1993,class_Copa América 1995,class_Copa América 1997,class_Copa América 1999,class_Copa América 2001,class_Copa América 2004,class_Copa América 2007,class_Copa América 2011,class_Copa América 2015,class_Copa América Centenario 2016,class_Copa América Centenario Play-In,class_Copa Centroamericana 2011,class_Copa Centroamericana 2013,class_Copa Centroamericana 2014,class_Copa Centroamericana 2017,class_EM-Qualifikation,class_EM-Qualifikation Playoffs,class_Europameisterschaft 1980,class_Europameisterschaft 1984,class_Europameisterschaft 1988,class_Europameisterschaft 1992,class_Europameisterschaft 1996,class_Europameisterschaft 2000,class_Europameisterschaft 2004,class_Europameisterschaft 2008,class_Europameisterschaft 2012,class_Europameisterschaft 2016,class_Freundschaftsspiele,class_Gold Cup 1991,class_Gold Cup 1993,class_Gold Cup 1996,class_Gold Cup 1998,class_Gold Cup 2000,class_Gold Cup 2002,class_Gold Cup 2003,class_Gold Cup 2005,class_Gold Cup 2007,class_Gold Cup 2009,class_Gold Cup 2011,class_Gold Cup 2013,class_Gold Cup 2015,class_Gold Cup 2017,class_Gold Cup Playoff,class_Gold Cup Qualifikation,class_König-Fahd-Pokal 1992,class_König-Fahd-Pokal 1995,class_OFC Nations Cup 1996,class_OFC Nat

In [11]:
#Validate: Is home-id of final Germany and away-id of final Argentina?
print(prepared_matches[prepared_matches.id_int_home == 3262].country_name_home.head(1))
print(prepared_matches[prepared_matches.id_int_home == 3437].country_name_home.head(1))

6    Germany
Name: country_name_home, dtype: object
100    Argentina
Name: country_name_home, dtype: object


In [13]:
# tell the models which dataframe columns are features and which one is the important label
features = list(prepared_matches_train_without_after_wm14.columns.values)
label = 'home_is_winner'

features.remove(label)

print(features)


['id_int', 'id_int_home', 'id_int_away', 'goals_home', 'goals_away', 'rank_home', 'total_points_home', 'previous_points_home', 'rank_change_home', 'cur_year_avg_home', 'cur_year_avg_weighted_home', 'last_year_avg_home', 'last_year_avg_weighted_home', 'two_year_ago_avg_home', 'two_year_ago_weighted_home', 'three_year_ago_avg_home', 'three_year_ago_weighted_home', 'rank_away', 'total_points_away', 'previous_points_away', 'rank_change_away', 'cur_year_avg_away', 'cur_year_avg_weighted_away', 'last_year_avg_away', 'last_year_avg_weighted_away', 'two_year_ago_avg_away', 'two_year_ago_weighted_away', 'three_year_ago_avg_away', 'three_year_ago_weighted_away', 'id_int_match', 'hp_1_id_int_player', 'hp_1_height', 'hp_1_country_id_int', 'hp_2_id_int_player', 'hp_2_height', 'hp_2_country_id_int', 'hp_3_id_int_player', 'hp_3_height', 'hp_3_country_id_int', 'hp_4_id_int_player', 'hp_4_height', 'hp_4_country_id_int', 'hp_5_id_int_player', 'hp_5_height', 'hp_5_country_id_int', 'hp_6_id_int_player', '

In [15]:
from sklearn.model_selection import train_test_split

all_X = prepared_matches_train[features]
all_y = prepared_matches_train[label]

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.00214, shuffle=False)

print('train', train_X.shape)
print('test', test_X.shape)

train (15711, 288)
test (34, 288)


In [46]:
DTree = DecisionTreeClassifier(criterion='entropy', 
                               max_depth=5,
                               min_samples_split=20,
                               min_samples_leaf=5)

DTree.fit(train_X, train_y)

predictions = DTree.predict(test_X)

accuracy = accuracy_score(test_y, predictions)

print(accuracy)

0.6470588235294118


In [47]:
list(zip(DTree.feature_importances_,features))

[(0.0, u'id_int'),
 (0.0909079715329537, u'id_int_home'),
 (0.2206943542815174, u'id_int_away'),
 (0.3449531433774878, 'rank_home'),
 (0.0, 'total_points_home'),
 (0.021179591860001786, 'previous_points_home'),
 (0.0, 'rank_change_home'),
 (0.0, 'cur_year_avg_home'),
 (0.0, 'cur_year_avg_weighted_home'),
 (0.0, 'last_year_avg_home'),
 (0.0, 'last_year_avg_weighted_home'),
 (0.0, 'two_year_ago_avg_home'),
 (0.0, 'two_year_ago_weighted_home'),
 (0.0, 'three_year_ago_avg_home'),
 (0.0, 'three_year_ago_weighted_home'),
 (0.23066157370925255, 'rank_away'),
 (0.0, 'total_points_away'),
 (0.01573787170880322, 'previous_points_away'),
 (0.0, 'rank_change_away'),
 (0.0, 'cur_year_avg_away'),
 (0.0, 'cur_year_avg_weighted_away'),
 (0.0, 'last_year_avg_away'),
 (0.0, 'last_year_avg_weighted_away'),
 (0.0, 'two_year_ago_avg_away'),
 (0.0, 'two_year_ago_weighted_away'),
 (0.0, 'three_year_ago_avg_away'),
 (0.0, 'three_year_ago_weighted_away'),
 (0.0, 'id_int_match'),
 (0.0, 'hp_1_id_int_player'),
 